In [1]:

!pip install pydal httpx[http2] trio ipywidgets chardet thefuzz 


import collections
import contextlib
import dbm
import json
import dill
import tqdm

print('working')
import pydal.objects
from thefuzz import fuzz 
import hashlib

try:
    import typedict
except ImportError:
    !pip install typedict
%run ~/work/__init__edwh__new.ipynb &2>&1>/dev/null 
from pydal import DAL, Field
import os, pathlib, httpx, trio
import typedict as typedict

if (workdir := pathlib.Path('/home/jovyan/work')).exists():
    os.chdir(workdir)
elif pathlib.Path('.jupyterlab/notebooks/DUO').exists():
    os.chdir('.jupyterlab/notebooks')
elif pathlib.Path('.').absolute().name == 'DUO':
    os.chdir('..')
elif pathlib.Path('.').absolute().name == 'notebooks':
    pass
else:
    raise ValueError(os.getcwd())
print(os.getcwd())

working
Python 3.10.10 | packaged by conda-forge | (main, Mar 24 2023, 20:08:06) [GCC 11.3.0]
Setting up for long running connection


/shared_code/edwh/core/pgcache/__init__.py:91: UserWarning: Magic aangeroepn, dat klopt nie. 
  warnings.warn("Magic aangeroepn, dat klopt nie. ")


 * postgresql://postgres@pgpool:5432/backend
Done.
/home/jovyan/work


In [2]:
import slugify
from functools import partial
import httpx, csv 
def fieldname(name):
    if name.isdigit():
        name = 'jaar_'+name
    return slugify.slugify(name, separator='_',replacements=[['-','_']])


In [3]:
sodk = httpx.get('https://redash.edwh.nl/api/queries/70/results.json?api_key=aVpyY7I0YkCPQ2kz3NoaiuIkJmjaOY4o62JhaNU0').json()['query_result']

In [4]:
sodk.keys()

dict_keys(['id', 'query_hash', 'query', 'data', 'data_source_id', 'runtime', 'retrieved_at'])

In [5]:
source_orgs = sodk['data']['rows']
print('scholen gevonden:',len(source_orgs))

scholen gevonden: 8869


In [6]:
_=y(source_orgs[1])

y|  source_orgs[1]:
        {'bisId': 678,
         'naam': 'Scholengemeenschap Spieringshoek voor Gymnasium Atheneum en '
                 'HAVO',
         'sanitizedNaam': 'scholengemeenschap-spieringshoek-voor-gymnasium-atheneum-en-havo',
         'onderwijsconcept': None,
         'adres': 'Van der Brugghenlaan 2',
         'postcode': '3118LA',
         'plaats': 'Schiedam',
         'plaatsSlug': 'schiedam',
         'onderwijssoortNiveau': 28,
         'latitude': 51.919487,
         'longitude': 4.378056,
         'aantalleerlingen': 982,
         'brinvests': '[{"instelling": "00JR", "vestiging": "00"}]',
         'denominatie': 'Katholiek',
         'denominatie_tip': 'Scholen voor katholiek onderwijs baseren hun '
                            'onderwijs op katholieke visies, gewoonten en '
                            'tradities.',
         'onderwijstype': 'VO',
         'onderwijstype_naam': 'Voortgezet onderwijs',
         'onderwijstype_tip': 'Na de basisschool gaat een le

In [7]:
db.organisation

<Table organisation (id, platform, effdt, effstatus, gid, prio, brin, vestigingscode, name, aka, website, country_code, coc, coc_location, phone, street, number, postalcode, city, correspondence_city, correspondence_street, correspondence_number, correspondence_postalcode, correspondence_country, tag_gid, lonlat, scholen_op_de_kaart_url, email, ceo_name, ceo_email, ceo_phone, quality_assurance_plan, aantekeningen, validated_by, validated_ts, education_type, sector, education_level, so_cluster, so_type, denomination, concept, student_count, subjects_with_specialised_teachers, board_gid, last_saved_by)>

In [8]:
%%sql
with o_effdt as 
(select gid, max(effdt), max(id) as id from organisation where effdt <= now() group by gid)
select * 
  from organisation o 
  inner join o_effdt on o.id = o_effdt.id
limit 10

 * postgresql://postgres@pgpool:5432/backend
10 rows affected.


id,platform,gid,name,coc,street,number,city,lonlat,tag_gid,validated_ts,validated_by,country_code,aka,website,email,scholen_op_de_kaart_url,aantekeningen,brin,vestigingscode,coc_location,phone,postalcode,correspondence_city,correspondence_street,correspondence_number,correspondence_postalcode,correspondence_country,ceo_name,ceo_email,ceo_phone,quality_assurance_plan,education_type,sector,education_level,so_cluster,so_type,denomination,concept,subjects_with_specialised_teachers,board_gid,is_active,effdt,effstatus,student_count,last_saved_by,prio,gid_1,max,id_1
19783,SvS,915dbbb1-84b0-4cb9-9409-0317b3f5d352,Vereniging Nutsschool Wassenaar,40408348,Zijllaan,39,WASSENAAR,"(4.4022312,52.1422118)",561537d7-d1ff-4f6d-a282-1be84eae1016,None,None,NL,None,None,None,None,None,04RT,00,None,None,None,None,None,None,None,None,None,None,None,None,r,|po|,|bo|,None,None,None,None,None,None,T,2023-02-13 16:17:21,T,233,Locatiegegevens aan Organisatie.ipynb op basis van leerlingen.sqlite/leerlingen.json and all_with_gid.db,100,915dbbb1-84b0-4cb9-9409-0317b3f5d352,2023-02-13 16:17:21,19783
18492,SvS,64103761-65f6-467f-8369-0e3248468d36,Berg en Bosch Onderwijs,30265539,Professor Bronkhorstlaan,22,BILTHOVEN,"(5.2099236,52.1529179)",f58e7f96-bc8d-43ee-adee-8047196d8861,None,None,NL,None,None,None,None,None,02RF,00,None,None,None,None,None,None,None,None,None,None,None,None,s,|so|vso|,None,None,None,None,None,None,None,T,2023-02-13 16:17:17,T,365,Locatiegegevens aan Organisatie.ipynb op basis van leerlingen.sqlite/leerlingen.json and all_with_gid.db,100,64103761-65f6-467f-8369-0e3248468d36,2023-02-13 16:17:17,18492
24943,SvS,342d9c43-5b17-4b37-ad26-15c827c83a5d,Openbare basischool De Zeevonk,34263476,Dr Jac P Thijsseweg,24,ZANDVOORT,"(4.541327,52.3793)",5f4d72fd-58d9-4dcb-9251-afe1942b5f68,None,None,NL,None,None,None,None,None,09RU,00,None,None,2041BM,None,None,None,None,None,None,None,None,None,r,|po|,|VO-SO|,None,None,Openbaar,None,None,None,T,2024-01-24 14:53:04.029074,T,232,Romy's scholenopdekaart data importer.ipynb (update_db_record),150,342d9c43-5b17-4b37-ad26-15c827c83a5d,2024-01-24 14:53:04.029074,24943
17966,SvS,533ad823-f7f5-460e-bed5-d3ec53cf8c03,Samenwerkingsschool De Diamant,1107731,Buorren,34-A,OUDEGA GEM SMALLINGERLND,"(5.9954931,53.1254223)",1d2d05b8-0ec9-4f28-a7aa-33b6edb92079,None,None,NL,None,None,None,None,None,06QJ,00,None,None,None,None,None,None,None,None,None,None,None,None,r,|po|,|bo|,None,None,None,None,None,None,T,2023-02-13 16:17:15,T,136,Locatiegegevens aan Organisatie.ipynb op basis van leerlingen.sqlite/leerlingen.json and all_with_gid.db,100,533ad823-f7f5-460e-bed5-d3ec53cf8c03,2023-02-13 16:17:15,17966
20145,SvS,9da21a16-bceb-48d3-909c-600c326a4d2a,School Met De Bijbel,30249028,Kerkstraat,7,RANDWIJK,"(5.7114077,51.954166)",cb278b3f-9e43-4f90-9e97-ebe5f54dff27,None,None,NL,None,None,None,None,None,05XQ,00,None,None,None,None,None,None,None,None,None,None,None,None,r,|po|,|bo|,None,None,None,None,None,None,T,2023-02-13 16:17:22,T,104,Locatiegegevens aan Organisatie.ipynb op basis van leerlingen.sqlite/leerlingen.json and all_with_gid.db,100,9da21a16-bceb-48d3-909c-600c326a4d2a,2023-02-13 16:17:22,20145
18612,SvS,685bce53-b879-4971-aaf9-976a603d4cad,Arembergschool,71767789,Julianastraat,2,ZWARTSLUIS,"(6.0693034,52.6441632)",778ce8c7-b98c-40b0-8e1c-1aaa8e112bd1,None,None,NL,None,None,None,None,None,03FU,00,None,None,None,None,None,None,None,None,None,None,None,None,r,|po|,|bo|,None,None,None,None,None,None,T,2023-02-13 16:17:17,T,144,Locatiegegevens aan Organisatie.ipynb op basis van leerlingen.sqlite/leerlingen.json and all_with_gid.db,100,685bce53-b879-4971-aaf9-976a603d4cad,2023-02-13 16:17:17,18612
24916,SvS,7ede95b3-c121-46b1-8019-88a3456cda6e,OBS J.H. Isings,8141209,Emperweg,70,EMPE,"(6.134757,52.153316)",030b73ab-2aef-4f5e-a8e3-271c73aeb40a,None,None,NL,None,None,None,None,None,12VJ,00,None,None,7399AG,None,None,None,None,None,None,None,None,None,r,|po|,|VO SO|,None,None,Openbaar,None,None,None,T,2024-01-24 14:52:4

In [30]:
def check_for_end_of_number(char, next_char):
    # Check if the current character is a space and the next character is a letter, if so it's the end of the number
    if char == ' ' and next_char.isalpha():
        return 'end_of_number'

def process_address(address, number, street, continue_number=True):
    # base case
    if not address:
        return number, street
    # Check if we should stop continuing the number, because we've reached the end of it
    if not continue_number:
        return number, street

    # Process the last character of the address
    last_char = address[-1]
    rest_of_address = address[:-1]

    # Update number and street
    # NOTE: number is reversed, so to have the correct order you need to add [::-1]
    number += last_char if continue_number else ""
    street = rest_of_address
    next_char = "" if not rest_of_address else rest_of_address[-1]

    # Classify the last character
    char_classification = check_for_end_of_number(last_char, next_char)

    # Update the continue_number flag based on classification
    if char_classification == 'end_of_number':
        continue_number = False

    # Continue processing the rest of the address
    return process_address(rest_of_address, number, street, continue_number)


def update_db_record(db_row_gid, api_row_data):
    """
    Update een database record met nieuwe waarden.

    Args:
        db_row_gid (str): GID van het database record.
        api_row_data (dict): Gegevens van het API-record.
    """
    last_db_row = db.executesql('''
        SELECT *
        FROM organisation
        WHERE gid = %s
        ORDER BY effdt DESC
        LIMIT 1
    ''', (db_row_gid,), as_dict=True)[0]
    new_values = {
        'name': api_row_data['api_row_naam'] if api_row_data['api_row_naam'] else last_db_row['name'],
        'concept': api_row_data['api_row_onderwijsconcept'] if api_row_data['api_row_onderwijsconcept'] else last_db_row['concept'],
        'street': api_row_data['api_row_street'] if api_row_data['api_row_street'] else last_db_row['street'],
        'number': api_row_data['api_row_number'] if api_row_data['api_row_number'] else last_db_row['number'],
        'postalcode': api_row_data['api_row_postcode'] if api_row_data['api_row_postcode'] else last_db_row['postalcode'],
        'city': api_row_data['api_row_plaats'] if api_row_data['api_row_plaats'] else last_db_row['city'],
        'brin': api_row_data['api_row_brin'] if api_row_data['api_row_brin'] else last_db_row['brin'],
        'vestigingscode': api_row_data['api_row_vestigingscode'] if api_row_data['api_row_vestigingscode'] else last_db_row['vestigingscode'],
        'lonlat': (api_row_data['api_row_longitude'], api_row_data['api_row_latitude']) if api_row_data['api_row_longitude'] is not None or api_row_data['api_row_latitude'] is not None else last_db_row['lonlat'],
        'sector': api_row_data['api_row_onderwijstype'], # er is geen None waarde, dus altijd overnemen
        # 'education_level': remove_weird_tokens(last_db_row['education_level']) if last_db_row['education_level'] else None,
        'education_level': tuple(level for level in last_db_row['education_level'].split('|') if level != '') if last_db_row['education_level'] else None,
        'subjects_with_specialised_teachers': tuple(level for level in last_db_row['subjects_with_specialised_teachers'].split('|') if level != '') if last_db_row['subjects_with_specialised_teachers'] else None,
        'student_count': api_row_data['api_row_aantalleerlingen'] if api_row_data['api_row_aantalleerlingen'] != 1 else last_db_row['student_count'],
        'effdt': 'NOW()',
        'denomination': api_row_data['api_row_denominatie'], # komt sowieso niet voor, dus altijd overnemen
        'prio': 150
    }
    db.organisation.sector.filter_in = lambda value: 'po' if value == 'BO' else value.lower()
    #db_record = db(db.organisation.gid == db_row_gid).select().first().as_dict()
    last_db_row.update(new_values)
    # remove ivm duplicate key value, dit moet een nieuwe row zijn
    del last_db_row['id']
    # deze nieuwe regel is niet gevalideerd, dus mogelijke waarden van validatie niet overnemen, default is null 
    del last_db_row['validated_by']
    del last_db_row['validated_ts']
    # aangegeven dat dit bestand de bron is. 
    last_db_row['last_saved_by'] = 'Romy\'s scholenopdekaart data importer.ipynb (update_db_record)'
    db.organisation.insert(**last_db_row)



def onderzoek_en_verzamel_gevonden_waarden(api_data):
    """
    Onderzoekt de meest voorkomende waarden in de 'brinvests' van de API-gegevens en verzamelt gevonden waarden.

    Args:
        api_data (list): Lijst van API-gegevens.

    Returns:
        Tuple: Tuple van (gevonden_waarden).
    """
    # Onderzoek de meest voorkomende waarden in de 'brinvests' van de API-gegevens
    onderzoek = collections.Counter([x['brinvests'] for x in api_data])
    meest_voorkomend = onderzoek.most_common()
    meest_voorkomend_minstens_2 = [(woord, aantal) for woord, aantal in meest_voorkomend if aantal >= 2]

    # Lijst om gevonden waarden op te slaan
    gevonden_waarden = []

    # Parse de JSON-geformatteerde strings en verzamel gevonden waarden
    for json_str, _ in meest_voorkomend_minstens_2:
        data_list = json.loads(json_str.replace("'", "\"")) if json_str else None
        if data_list:
            gevonden_waarden.extend([(item, _) for item in data_list])

    # Houd het aantal per combinatie bij
    aantal_per_combinatie = collections.Counter()

    # Handmatig het aantal bijhouden
    for item, aantal in gevonden_waarden:
        db_row_brin = item['instelling']
        db_row_vestigingscode = item['vestiging']
        aantal_per_combinatie[(db_row_brin, db_row_vestigingscode)] += aantal

    return gevonden_waarden, aantal


def rows_requirements(api_row, db_row=None):
    api_row_brinvests_in_json = json.loads(api_row['brinvests'])
    for api_brinvests in api_row_brinvests_in_json:
        api_row_brinvest = api_brinvests['instelling'].upper().strip() + api_brinvests['vestiging'].upper().strip() if api_brinvests['instelling'] or api_brinvests['vestiging'] else 'None_api_brinvests'
        api_row_brin = api_brinvests['instelling'].upper().strip() if api_brinvests['instelling'] else None
        api_row_vestigingscode = api_brinvests['vestiging'].upper().strip() if api_brinvests['vestiging'] else None
        api_row_naam = api_row['naam']
        api_row_postcode = api_row['postcode'].replace(' ', '').upper() if api_row['postcode'] else None
        api_row_adres = api_row['adres'] if api_row['adres'] else None
        api_row_number, api_row_street = process_address(api_row_adres, "", "")
        api_row_number = api_row_number[::-1].strip()
        api_row_plaats = api_row['plaats'].upper()
        api_row_longitude = api_row['longitude']
        api_row_latitude = api_row['latitude']
        api_row_aantalleerlingen = api_row['aantalleerlingen'] if api_row['aantalleerlingen'] else None
        api_row_onderwijsconcept = api_row['onderwijsconcept'] if api_row['onderwijsconcept'] else None
        api_row_bisid = api_row['bisId']
        api_row_denominatie = api_row['denominatie']
        api_row_onderwijstype = api_row['onderwijstype']
        db_row_brinvest = None
        db_row_brin = None
        db_row_vestigingscode = None
        db_row_gid = None
        combinatie = None
        if db_row is not None:
            db_row_brinvest = db_row['brin'].upper().strip() + db_row['vestigingscode'].upper().strip() if db_row['brin'] or db_row['vestigingscode'] else 'None_db_brinvest'
            db_row_brin = db_row['brin'].upper().strip() if db_row['brin'] else "None_db_brin"
            db_row_vestigingscode = db_row['vestigingscode'].upper().strip() if db_row['vestigingscode'] else "None_db_vestigingscode"
            db_row_gid = db_row['gid']
            combinatie = (db_row_brin, db_row_vestigingscode)
            

        return locals()
    
# Een lijst met dictionaries die de databasegegevens bevatten.
db_data = db.executesql('''
        WITH o_effdt AS (
            SELECT gid, MAX(effdt) AS max_effdt, MAX(id) AS id
            FROM organisation
            WHERE effdt <= NOW()
            GROUP BY gid
        )
        SELECT o.gid, o.name, o.concept, o.street, o.number, o.postalcode, o.city, o.brin, o.vestigingscode, o.lonlat, o.student_count
        FROM organisation AS o
        INNER JOIN o_effdt ON o.id = o_effdt.id
        WHERE o.effstatus = 'T'
    ''', as_dict=True)

# API-gegevens
api_data = source_orgs

# Gebruik van de functie om dubbele brinvest waardes eruit te vissen om later te kijken welke het beste ratio heeft vergeleken de brinvest uit de db
dubbele_brinvest_waardes = onderzoek_en_verzamel_gevonden_waarden(api_data)
dubbele_brinvest_list = []
best_match_brinvest_dict = {}
for dubbele_brinvest_waarde in dubbele_brinvest_waardes[0]:
    (brinvest_dict, aantal) = dubbele_brinvest_waarde
    brinvest = brinvest_dict['instelling'] + brinvest_dict['vestiging'] if brinvest_dict['instelling'] and brinvest_dict['vestiging'] else 'None_brinvest'
    dubbele_brinvest_list.append(brinvest)

# Verzamel unieke waarden van de sleutels uit de database
handled_values_for_duplicates_db = set()
handled_values_for_duplicates_api = set()

In [63]:
db.rollback()

In [64]:
max_before_inserts = y(db(db.organisation).select(db.organisation.id.max()).first()[db.organisation.id.max()])

y|  db(db.organisation).select(db.organisation.id.max()).first()[db.organisation.id.max()]: 25051


In [31]:
for db_row in tqdm(db_data):
    for api_row in api_data:
        pair = (api_row['bisId'], api_row['onderwijstype'])
        double_brinvest_dict_temp = {'ratio' : set(), 'row_requirements': []}
        for row_requirements in [rows_requirements(api_row, db_row)]:
            if row_requirements['db_row_brinvest'] == row_requirements['api_row_brinvest'] and not row_requirements['db_row_gid'] in handled_values_for_duplicates_db and not pair in handled_values_for_duplicates_api:
                if row_requirements['api_row_brinvest'] in dubbele_brinvest_list:
                    naam_ratio = fuzz.ratio(db_row['name'].lower(), row_requirements['api_row_naam'].lower())
                    plaats_ratio = fuzz.ratio(db_row['city'].lower().split()[0] if db_row['city'] else 'None_db_city', row_requirements['api_row_plaats'].lower().split()[0] if row_requirements['api_row_plaats'] else 'None_api_plaats')
                    adres_ratio = fuzz.ratio(db_row['street'].lower(), row_requirements['api_row_street'].lower())
                    add_together_ratio = (naam_ratio + plaats_ratio + adres_ratio)
                    double_brinvest_dict_temp['ratio'] = add_together_ratio
                    double_brinvest_dict_temp['row_requirements'] = [row_requirements]
                    brinvest_key = row_requirements['api_row_brinvest']
                    if brinvest_key not in best_match_brinvest_dict:
                        best_match_brinvest_dict[brinvest_key] = double_brinvest_dict_temp
                    else:
                        existing_brinvest_key = best_match_brinvest_dict[brinvest_key]
                        existing_ratio = existing_brinvest_key['ratio']
                        new_ratio = double_brinvest_dict_temp['ratio']
                        if new_ratio > existing_ratio:
                            (best_match_brinvest_dict.
                             update({brinvest_key: {'ratio': double_brinvest_dict_temp['ratio'], 'row_requirements': double_brinvest_dict_temp['row_requirements']}}))
                else:
                    update_db_record(row_requirements['db_row_gid'], row_requirements)
                    handled_values_for_duplicates_db.add(row_requirements['db_row_gid'])
                    handled_values_for_duplicates_api.add(pair)
                    break


for brinvest_key, ratio_and_row_requirements in tqdm(best_match_brinvest_dict.items()):
    for row_requirement in ratio_and_row_requirements['row_requirements']:
        pair = (row_requirement['api_row']['bisId'], row_requirement['api_row']['onderwijstype'])
        db_row_gid = row_requirement['db_row']['gid']
        if db_row_gid not in handled_values_for_duplicates_db and pair not in handled_values_for_duplicates_api:
            update_db_record(db_row_gid, row_requirement)
            handled_values_for_duplicates_db.add(db_row_gid)
            handled_values_for_duplicates_api.add(pair)


for api_row in tqdm(api_data):
    pair = (api_row['bisId'], api_row['onderwijstype'])
    for row_requirements in [rows_requirements(api_row)]:
        if pair not in handled_values_for_duplicates_api:
            new_values = {
                    'gid': uuid.uuid4(),
                    'name': row_requirements['api_row_naam'],
                    'concept': row_requirements['api_row_onderwijsconcept'],
                    'street': row_requirements['api_row_street'],
                    'number': row_requirements['api_row_number'],
                    'postalcode': row_requirements['api_row_postcode'],
                    'city': row_requirements['api_row_plaats'],
                    'brin': row_requirements['api_row_brin'],
                    'vestigingscode': row_requirements['api_row_vestigingscode'],
                    'lonlat': (row_requirements['api_row_longitude'], row_requirements['api_row_latitude']) if row_requirements['api_row_longitude'] is not None and row_requirements['api_row_latitude'] is not None else None,
                    'sector': row_requirements['api_row_onderwijstype'],
                    'student_count': row_requirements['api_row_aantalleerlingen'],
                    'effdt': 'NOW()',
                    'prio': 150, 
                    'denomination': row_requirements['api_row_denominatie'],
                    'last_saved_by': 'Romy\'s scholenopdekaart data importer.ipynb (nieuwe scholen)'
                }
                
            db.organisation.lonlat.filter_in = lambda value: (value[0], value[1]) if value is not None else None
            db.organisation.sector.filter_in = lambda value: 'po' if value == 'BO' else value.lower()
            db.organisation.insert(**new_values)
            handled_values_for_duplicates_api.add(pair)
            break
print('klaar')

100%|██████████| 8869/8869 [00:03<00:00, 2652.18it/s]

klaar


In [32]:
db.commit()

In [ ]:
handled_values_for_duplicates_api_dict = set()
count = 0

for api_row in api_data:
    pair = (api_row['bisId'], api_row['onderwijstype'])
    if pair not in handled_values_for_duplicates_api_dict:
        handled_values_for_duplicates_api_dict.add(pair)
    else:
        count += 1
    if api_row['bisId'] == 23250:
        print(api_row)

print(len(handled_values_for_duplicates_api_dict))
